## 주조 공정최적화 AI 데이터셋
### 군집화와 차원축소
https://www.kamp-ai.kr/aidataDetail?AI_SEARCH=%EC%A3%BC%EC%A1%B0+%EA%B3%B5%EC%A0%95%EC%B5%9C%EC%A0%81%ED%99%94+AI+%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B&page=1&DATASET_SEQ=53&DISPLAY_MODE_SEL=CARD&EQUIP_SEL=&GUBUN_SEL=&FILE_TYPE_SEL=&WDATE_SEL=

출처 : 중소벤처기업부, Korea AI Manufacturing Platform(KAMP)

In [23]:
import koreanize_matplotlib

In [ ]:
import pandas as pd

# 데이터 불러오기 (Unnamed: 0 컬럼을 인덱스로 지정)
df = pd.read_csv('data/casting.csv', encoding='cp949', index_col='Unnamed: 0')
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='object')

In [ ]:
df.hist(figsize=(10, 10), bins=50);

In [ ]:

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import numpy as np
import warnings

# 수치형 데이터만 추출 (이상치 및 무한대/NaN 값 제거)
numeric_df = df.select_dtypes(include=['number']).replace([np.inf, -np.inf], np.nan).dropna()

# 모든 컬럼이 0이거나 상수인 경우 KMeans에서 오류 발생하므로, 분산이 0인 컬럼 제거
numeric_df = numeric_df.loc[:, numeric_df.std() > 0]

# 표준화
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_df)

# KMeans 군집화 (5개 군집)
# n_init 값을 명시적으로 지정 (sklearn 1.4 이상에서 경고 방지)
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
clusters = kmeans.fit_predict(scaled_data)

# 원본 데이터프레임에 군집 결과 추가
df['cluster'] = -1
df.loc[numeric_df.index, 'cluster'] = clusters

# 각 군집별 데이터 개수 출력
print(df['cluster'].value_counts())


In [ ]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score, silhouette_samples

# sklearn의 RuntimeWarning 무시 (divide by zero, overflow 등)
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    # 실루엣 점수 계산
    sil_score = silhouette_score(scaled_data, clusters)
    print(f"전체 실루엣 점수: {sil_score:.4f}")

    # 각 샘플별 실루엣 계수
    sample_silhouette_values = silhouette_samples(scaled_data, clusters)

n_clusters = 5
fig, ax1 = plt.subplots(1, 1)
fig.set_size_inches(8, 6)

y_lower = 10
for i in range(n_clusters):
    ith_cluster_silhouette_values = sample_silhouette_values[clusters == i]
    ith_cluster_silhouette_values = ith_cluster_silhouette_values[~np.isnan(ith_cluster_silhouette_values)]
    ith_cluster_silhouette_values.sort()
    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = plt.cm.nipy_spectral(float(i) / n_clusters)
    ax1.fill_betweenx(np.arange(y_lower, y_upper),
                      0, ith_cluster_silhouette_values,
                      facecolor=color, edgecolor=color, alpha=0.7)

    ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10  # 10 for spacing between clusters

ax1.set_title("군집별 실루엣 계수 시각화")
ax1.set_xlabel("실루엣 계수")
ax1.set_ylabel("샘플 인덱스")

ax1.axvline(x=sil_score, color="red", linestyle="--", label="전체 실루엣 점수")
ax1.set_yticks([])
ax1.set_xlim([-0.1, 1])
ax1.legend()
plt.show()


In [ ]:
from sklearn.decomposition import PCA

# PCA 변환 전에 NaN, inf, -inf 값이 있으면 0으로 대체 (권장 방식)
import numpy as np
scaled_data_clean = np.nan_to_num(scaled_data, nan=0.0, posinf=0.0, neginf=0.0)

# scaled_data를 2차원으로 차원 축소
pca = PCA(n_components=2)
data_2d = pca.fit_transform(scaled_data_clean)

plt.figure(figsize=(8, 6))
for i in range(n_clusters):
    plt.scatter(
        data_2d[clusters == i, 0],
        data_2d[clusters == i, 1],
        label=f'Cluster {i}',
        alpha=0.6
    )
plt.title('PCA 기반 2차원 군집 시각화')
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.legend()
plt.show()


In [ ]:
# 군집 결과 해석을 위한 대표적인 분석 방법: 각 군집별 주요 변수의 평균/분포 비교, 군집별 특성 파악

# 1. 군집별 주요 변수의 평균값 비교

# clusters의 길이와 df의 길이가 다를 경우, clusters가 어떤 행에 해당하는지 인덱스를 명확히 지정해야 함
import numpy as np

# 매우 중요: 전처리된 데이터프레임의 인덱스를 'valid_idx'에 저장합니다.
valid_idx = numeric_df.index

# clusters가 numpy array 또는 pandas Series라고 가정
if len(clusters) != len(df):
    # scaled_data와 df의 인덱스가 다를 수 있으므로, scaled_data를 만든 원본 df의 인덱스를 추적해야 함
    # scaled_data를 만들 때 사용한 인덱스가 valid_idx라면, 그 인덱스를 사용
    # 예시: valid_idx = df.dropna(subset=사용한_컬럼들).index
    # 아래는 일반적인 예시 코드 (valid_idx가 있다고 가정)
    try:
        valid_idx
    except NameError:
        raise ValueError(
            "clusters와 df의 길이가 다릅니다. "
            "scaled_data를 만들 때 사용한 원본 df의 인덱스(valid_idx)를 지정해 주세요.\n"
            "예시: valid_idx = ...; clustered_df = df.loc[valid_idx].copy(); clustered_df['cluster'] = clusters"
        )
    clustered_df = df.loc[valid_idx].copy()
    clustered_df['cluster'] = clusters
else:
    clustered_df = df.copy()
    clustered_df['cluster'] = clusters

# 주요 수치형 변수만 추출 (예시: scaled_data의 컬럼명 사용)
numeric_cols = clustered_df.select_dtypes(include=[np.number]).columns

# 군집별 평균값
cluster_means = clustered_df.groupby('cluster')[numeric_cols].mean()
print("군집별 주요 변수 평균값:")
from IPython.display import display  # display 함수가 없을 수 있으니 명시적으로 import
display(cluster_means)

# 2. 군집별 변수 분포 시각화 (박스플롯)
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(15, 6))
for i, col in enumerate(numeric_cols[:5]):  # 주요 변수 5개만 예시로 시각화
    plt.subplot(1, 5, i+1)
    sns.boxplot(x='cluster', y=col, data=clustered_df)
    plt.title(col)
    plt.xlabel('Cluster')
    plt.tight_layout()
plt.suptitle('군집별 주요 변수 분포 (일부 변수)')
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

# 3. 군집별 샘플 수 확인
print("군집별 샘플 수:")
print(clustered_df['cluster'].value_counts().sort_index())

# 4. (선택) 군집별 범주형 변수 분포 확인
cat_cols = clustered_df.select_dtypes(include=['object']).columns
if len(cat_cols) > 0:
    for col in cat_cols[:2]:  # 범주형 변수 2개만 예시
        print(f"\n군집별 {col} 분포:")
        display(clustered_df.groupby('cluster')[col].value_counts(normalize=True).unstack().fillna(0))
# 일반적으로 scaled_data를 만들 때 사용한 주요 수치형 컬럼에서 결측치가 없는 행을 사용합니다.
# 예시로 scaled_data와 동일한 컬럼을 사용했다고 가정하고, 주요 수치형 변수 5개를 사용합니다.

# 주요 수치형 변수 5개를 사용하여 결측치가 없는 인덱스를 valid_idx로 지정
main_numeric_cols = clustered_df.select_dtypes(include=[np.number]).columns[:5]
valid_idx = df.dropna(subset=main_numeric_cols).index

print("자동으로 추정한 valid_idx (결측치 없는 행의 인덱스, 주요 변수 5개 기준):")
print(valid_idx)
